In [8]:
import gzip
import json
import nltk
import pandas as pd

from gensim import corpora
from gensim.parsing import preprocessing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm

tqdm.pandas()

nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/adrian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/adrian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
stop_words = {
    "spanish": stopwords.words("spanish")
}
# Usamos solamente Español a fines de utilizar menos recursos de computación.
#,
#    "portuguese": stopwords.words("portuguese")
#}

In [10]:
data = []

for language in tqdm(stop_words.keys()):
    for split in tqdm(["train", "test", "validation"]):
        df = pd.read_json(f"../data/meli-challenge-2019/{language}.{split}.jsonl.gz", lines=True)
        data.append(df)

data = pd.concat(data, ignore_index=True)
data.head()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

,language,label_quality,title,category,split,tokenized_title,data,target,n_labels,size
0,spanish,reliable,Casita Muñecas Barbies Pintadas,DOLLHOUSES,train,"[casita, muñecas, barbies, pintadas]","[50001, 2, 50000, 3]",0,632,4895280
1,spanish,unreliable,Neceser Cromado Holográfico,TOILETRY_BAGS,train,"[neceser, cromado, holográfico]","[6, 4, 5]",1,632,4895280
2,spanish,unreliable,Funda Asiento A Medida D20 Chevrolet,CAR_SEAT_COVERS,train,"[funda, asiento, medida, chevrolet]","[9, 7, 10, 8]",2,632,4895280
3,spanish,unreliable,Embrague Ford Focus One 1.8 8v Td (90cv) Desde...,AUTOMOTIVE_CLUTCH_KITS,train,"[embrague, ford, focus, one]","[11, 13, 12, 14]",3,632,4895280
4,spanish,unreliable,Bateria Panasonic Dmwbcf10 Lumix Dmc-fx60n Dmc...,CAMERA_BATTERIES,train,"[bateria, panasonic, dmwbcf, lumix, dmc, fxn, ...","[15, 19, 17, 18, 16, 1, 1, 1]",4,632,4895280


In [11]:
def clean_titles(row):
    title = preprocessing.strip_tags(row["title"].lower())
    title = preprocessing.strip_punctuation(title)
    title = preprocessing.strip_numeric(title)
    title = word_tokenize(title, language=row["language"])
    title = [word for word in title if word not in stop_words[row["language"]]]
    title = [word for word in title if len(word) >= 3]
    return title

data["tokenized_title"] = data.progress_apply(clean_titles, axis=1)

  0%|          | 0/6182780 [00:00<?, ?it/s]

In [12]:
for language, lang_df in data.groupby(["language"]):
    dictionary = corpora.Dictionary(lang_df["tokenized_title"].tolist())
    dictionary.filter_extremes(no_below=2, no_above=1, keep_n=50000)
    dictionary.compactify()
    dictionary.patch_with_special_tokens({
        "[PAD]": 0,
        "[UNK]": 1
    })
    
    data.loc[lang_df.index, "data"] = lang_df["tokenized_title"].progress_map(
        lambda t: dictionary.doc2idx(
            document=t,
            unknown_word_index=1
        )
    )
    
    label_to_target = {label: index for index, label in enumerate(lang_df["category"].unique())}
    data.loc[lang_df.index, "target"] = lang_df["category"].progress_map(lambda l: label_to_target[l])
    
    with gzip.open(f"../data/meli-challenge-2019/{language}_token_to_index.json.gz", "wt") as fh:
        json.dump(dictionary.token2id, fh)

data.head()

  0%|          | 0/6182780 [00:00<?, ?it/s]

  0%|          | 0/6182780 [00:00<?, ?it/s]

,language,label_quality,title,category,split,tokenized_title,data,target,n_labels,size
0,spanish,reliable,Casita Muñecas Barbies Pintadas,DOLLHOUSES,train,"[casita, muñecas, barbies, pintadas]","[50001, 2, 50000, 3]",0,632,4895280
1,spanish,unreliable,Neceser Cromado Holográfico,TOILETRY_BAGS,train,"[neceser, cromado, holográfico]","[6, 4, 5]",1,632,4895280
2,spanish,unreliable,Funda Asiento A Medida D20 Chevrolet,CAR_SEAT_COVERS,train,"[funda, asiento, medida, chevrolet]","[9, 7, 10, 8]",2,632,4895280
3,spanish,unreliable,Embrague Ford Focus One 1.8 8v Td (90cv) Desde...,AUTOMOTIVE_CLUTCH_KITS,train,"[embrague, ford, focus, one]","[11, 13, 12, 14]",3,632,4895280
4,spanish,unreliable,Bateria Panasonic Dmwbcf10 Lumix Dmc-fx60n Dmc...,CAMERA_BATTERIES,train,"[bateria, panasonic, dmwbcf, lumix, dmc, fxn, ...","[15, 19, 17, 18, 16, 1, 1, 1]",4,632,4895280


In [13]:
n_labels = data.groupby(["language"])["target"].max().to_dict()
n_labels

{'spanish': 631}

In [14]:
split_size = data.groupby(["language", "split"]).size().to_dict()
split_size

{('spanish', 'test'): 63680,
 ('spanish', 'train'): 4895280,
 ('spanish', 'validation'): 1223820}

In [15]:
data["n_labels"] = data.apply(lambda r: n_labels[r["language"]] + 1, axis=1)
data["size"] = data.apply(lambda r: split_size[(r["language"], r["split"])], axis=1)
data.head()

,language,label_quality,title,category,split,tokenized_title,data,target,n_labels,size
0,spanish,reliable,Casita Muñecas Barbies Pintadas,DOLLHOUSES,train,"[casita, muñecas, barbies, pintadas]","[50001, 2, 50000, 3]",0,632,4895280
1,spanish,unreliable,Neceser Cromado Holográfico,TOILETRY_BAGS,train,"[neceser, cromado, holográfico]","[6, 4, 5]",1,632,4895280
2,spanish,unreliable,Funda Asiento A Medida D20 Chevrolet,CAR_SEAT_COVERS,train,"[funda, asiento, medida, chevrolet]","[9, 7, 10, 8]",2,632,4895280
3,spanish,unreliable,Embrague Ford Focus One 1.8 8v Td (90cv) Desde...,AUTOMOTIVE_CLUTCH_KITS,train,"[embrague, ford, focus, one]","[11, 13, 12, 14]",3,632,4895280
4,spanish,unreliable,Bateria Panasonic Dmwbcf10 Lumix Dmc-fx60n Dmc...,CAMERA_BATTERIES,train,"[bateria, panasonic, dmwbcf, lumix, dmc, fxn, ...","[15, 19, 17, 18, 16, 1, 1, 1]",4,632,4895280


In [16]:
for (language, split), sub_df in data.groupby(["language", "split"]):
    sub_df.to_json(
        f"../data/meli-challenge-2019/{language}.{split}.jsonl.gz",
        lines=True,
        orient="records"
    )